# Data Analytics with PySpark - SQL Queries

#### Imports and PySpark Init

In [1]:
from pathlib import Path
import sys

In [2]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [3]:
conf = SparkConf() \
    .setAppName('pySparkExamples') \
    .setMaster('local')

sc = SparkContext(conf=conf)
spark = SparkSession(sc)

### Reading Dataset

#### Input paths and variables

In [4]:
PROJECT_ROOT = Path.cwd().parent.as_posix()
csv_file = f"{PROJECT_ROOT}/data/emp_details.json"
uri_scheme = "file://"

####  File Schema

In [5]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

schema = StructType([
    StructField("userId", StringType(), True),
    StructField("jobTitle", StringType(), True),
    StructField("firstName", StringType(), True),
    StructField("lastName", StringType(), True),
    StructField("employeeCode", StringType(), True),
    StructField("region", StringType(), True),
    StructField("phoneNumber", StringType(), True),
    StructField("emailAddress", StringType(), True),
    StructField("salary", IntegerType(), True)
])

#### Read file

In [6]:
input_path = f"{uri_scheme}/{csv_file}" if sys.platform == "win32" else f"{uri_scheme}{csv_file}"
df = spark.read.json(input_path, schema=schema)

# Required for executing SQL Queries
df.createOrReplaceTempView("Employees")

In [7]:
df.show()

+------+-----------------+-----------+--------+------------+------+-----------+-------------+------+
|userId|         jobTitle|  firstName|lastName|employeeCode|region|phoneNumber| emailAddress|salary|
+------+-----------------+-----------+--------+------------+------+-----------+-------------+------+
| cpoul|        Developer|Christopher|  Poulin|          E1|    US|     123456|cpoul@amey.me|100000|
| drome|        Developer|      David|    Rome|          E1|    US|    1111111|drome@amey.me|110000|
| tjohn|        Developer|        Tin| Johnson|          E3|    US|    2222222|tjohn@amey.me|110000|
| akolh|        Developer|       Amey|   Kolhe|          E1|    IN|     123456|akolh@amey.me|125000|
| hdwye|   Senior Manager|      Helen|   Dwyer|          E5|    US|    2222222|hdwye@amey.me|150000|
|  ajha|   Senior Manager|       Amit|     Jha|          E6|    IN|    2222222| ajha@amey.me|160000|
| kkaga|   Vice President|   Kristina| Kaganer|          E7|    US|    2222222|kkaga@amey.m

## SQL Queries

### Query 1

#### Statement: Find distinct jobTitle

In [8]:
result_df = df.select("jobTitle").distinct()
result_df.show()

+-----------------+
|         jobTitle|
+-----------------+
|   Vice President|
|        Developer|
|Data Science Lead|
| Business Analyst|
|   Senior Manager|
+-----------------+



In [9]:
sql_result = spark.sql("SELECT distinct jobTitle from Employees")
result_df.show()

+-----------------+
|         jobTitle|
+-----------------+
|   Vice President|
|        Developer|
|Data Science Lead|
| Business Analyst|
|   Senior Manager|
+-----------------+



### Query 2

#### Statement: Get all employee details where region is US

In [10]:
result_df = df.filter(F.col("region")=="US")
result_df.show()

+------+--------------+-----------+--------+------------+------+-----------+-------------+------+
|userId|      jobTitle|  firstName|lastName|employeeCode|region|phoneNumber| emailAddress|salary|
+------+--------------+-----------+--------+------------+------+-----------+-------------+------+
| cpoul|     Developer|Christopher|  Poulin|          E1|    US|     123456|cpoul@amey.me|100000|
| drome|     Developer|      David|    Rome|          E1|    US|    1111111|drome@amey.me|110000|
| tjohn|     Developer|        Tin| Johnson|          E3|    US|    2222222|tjohn@amey.me|110000|
| hdwye|Senior Manager|      Helen|   Dwyer|          E5|    US|    2222222|hdwye@amey.me|150000|
| kkaga|Vice President|   Kristina| Kaganer|          E7|    US|    2222222|kkaga@amey.me|200000|
+------+--------------+-----------+--------+------------+------+-----------+-------------+------+



In [11]:
sql_result = spark.sql("SELECT * from Employees WHERE region = 'US'")
result_df.show()

+------+--------------+-----------+--------+------------+------+-----------+-------------+------+
|userId|      jobTitle|  firstName|lastName|employeeCode|region|phoneNumber| emailAddress|salary|
+------+--------------+-----------+--------+------------+------+-----------+-------------+------+
| cpoul|     Developer|Christopher|  Poulin|          E1|    US|     123456|cpoul@amey.me|100000|
| drome|     Developer|      David|    Rome|          E1|    US|    1111111|drome@amey.me|110000|
| tjohn|     Developer|        Tin| Johnson|          E3|    US|    2222222|tjohn@amey.me|110000|
| hdwye|Senior Manager|      Helen|   Dwyer|          E5|    US|    2222222|hdwye@amey.me|150000|
| kkaga|Vice President|   Kristina| Kaganer|          E7|    US|    2222222|kkaga@amey.me|200000|
+------+--------------+-----------+--------+------------+------+-----------+-------------+------+



### Query 3
#### Statement: Find min, max and average salaries for each jobTitle

In [12]:
result_df = df.groupby(["jobTitle"]) \
    .agg(
        F.min("salary").alias("min_salary"), 
        F.max("salary").alias("max_salary"), 
        F.avg("salary").alias("avg_salary")
)
result_df.show()

+-----------------+----------+----------+----------+
|         jobTitle|min_salary|max_salary|avg_salary|
+-----------------+----------+----------+----------+
|   Vice President|    200000|    200000|  200000.0|
|        Developer|    100000|    125000|  111250.0|
|Data Science Lead|    175000|    175000|  175000.0|
| Business Analyst|    175000|    175000|  175000.0|
|   Senior Manager|    150000|    160000|  155000.0|
+-----------------+----------+----------+----------+



In [13]:
sql_result = spark.sql("SELECT jobTitle, min(salary), max(salary), avg(salary) from Employees GROUP BY jobTitle")
sql_result.show()

+-----------------+-----------+-----------+-----------+
|         jobTitle|min(salary)|max(salary)|avg(salary)|
+-----------------+-----------+-----------+-----------+
|   Vice President|     200000|     200000|   200000.0|
|        Developer|     100000|     125000|   111250.0|
|Data Science Lead|     175000|     175000|   175000.0|
| Business Analyst|     175000|     175000|   175000.0|
|   Senior Manager|     150000|     160000|   155000.0|
+-----------------+-----------+-----------+-----------+



### Query 4
#### Statement: Increase Salaries of Developers by 10%

In [14]:
result_df = df.withColumn("salary", F.when(F.col("jobTitle") == "Developer", F.col("salary")* 1.10).otherwise(F.col("salary")))
result_df.show()

+------+-----------------+-----------+--------+------------+------+-----------+-------------+------------------+
|userId|         jobTitle|  firstName|lastName|employeeCode|region|phoneNumber| emailAddress|            salary|
+------+-----------------+-----------+--------+------------+------+-----------+-------------+------------------+
| cpoul|        Developer|Christopher|  Poulin|          E1|    US|     123456|cpoul@amey.me|110000.00000000001|
| drome|        Developer|      David|    Rome|          E1|    US|    1111111|drome@amey.me|121000.00000000001|
| tjohn|        Developer|        Tin| Johnson|          E3|    US|    2222222|tjohn@amey.me|121000.00000000001|
| akolh|        Developer|       Amey|   Kolhe|          E1|    IN|     123456|akolh@amey.me|          137500.0|
| hdwye|   Senior Manager|      Helen|   Dwyer|          E5|    US|    2222222|hdwye@amey.me|          150000.0|
|  ajha|   Senior Manager|       Amit|     Jha|          E6|    IN|    2222222| ajha@amey.me|   

In [15]:
sql_result = spark.sql("SELECT userId, jobTitle, firstName, lastName, employeeCode, region, phoneNumber, emailAddress, salary * 1.1 as salary from Employees")
sql_result.show()

+------+-----------------+-----------+--------+------------+------+-----------+-------------+--------+
|userId|         jobTitle|  firstName|lastName|employeeCode|region|phoneNumber| emailAddress|  salary|
+------+-----------------+-----------+--------+------------+------+-----------+-------------+--------+
| cpoul|        Developer|Christopher|  Poulin|          E1|    US|     123456|cpoul@amey.me|110000.0|
| drome|        Developer|      David|    Rome|          E1|    US|    1111111|drome@amey.me|121000.0|
| tjohn|        Developer|        Tin| Johnson|          E3|    US|    2222222|tjohn@amey.me|121000.0|
| akolh|        Developer|       Amey|   Kolhe|          E1|    IN|     123456|akolh@amey.me|137500.0|
| hdwye|   Senior Manager|      Helen|   Dwyer|          E5|    US|    2222222|hdwye@amey.me|165000.0|
|  ajha|   Senior Manager|       Amit|     Jha|          E6|    IN|    2222222| ajha@amey.me|176000.0|
| kkaga|   Vice President|   Kristina| Kaganer|          E7|    US|    22